In [1]:
import re
import numpy as np
from sklearn import datasets
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.svm import SVC


class SVM(object):

    def __init__(self, training_dataset_, test_dataset_):
        self.training_dataset = training_dataset_
        self.test_dataset = test_dataset_
        self.classes = {}
        self.X_train = None
        self.Y_train = None
        self.X_test = None
        self.Y_test = None

        self.support_indices = None
        self.train_errors = None
        self.test_errors = None
        self.train_loss = None
        self.test_loss = None

    def read_data(self):
        f = open(self.training_dataset, 'r')
        rows = list(re.split(' ', row) for row in re.split('\n', f.read())[:-1])
        names, self.Y_train = np.unique(list(row[-1] for row in rows), return_inverse=True)
        self.X_train = np.empty((0,4), float)
        f.close()
        for row in rows:
            self.X_train = np.append(self.X_train, np.array([np.array(row[:-1]).astype(float)]), axis = 0)
        f = open(self.test_dataset, 'r')
        f.close()

        f = open(self.test_dataset, 'r')
        rows = list(re.split(' ', row) for row in re.split('\n', f.read())[:-1])
        names, self.Y_test = np.unique(list(row[-1] for row in rows), return_inverse=True)
        self.X_test = np.empty((0,4), float)
        f.close()
        for row in rows:
            self.X_test = np.append(self.X_test, np.array([np.array(row[:-1]).astype(float)]), axis = 0)

    # Using one-vs-rest strategy with SVC
    def SVM(self, C=1e5, kernel="linear", degree=1, gamma=1):

        # First, we should manually label the other two classes into one
        y_train_0 = []
        y_train_1 = []
        y_train_2 = []
        for i in self.Y_train:
            if (i == 0):
                y_train_0.append(i)
                y_train_1.append(-1)
                y_train_2.append(-1)
            elif (i == 1):
                y_train_0.append(-1)
                y_train_1.append(i)
                y_train_2.append(-1)
            elif (i == 2):
                y_train_0.append(-1)
                y_train_1.append(-1)
                y_train_2.append(i)
            else:
                break

        # Create model
        model_0 = SVC(C=C, kernel=kernel, decision_function_shape='ovo', degree=degree, gamma=gamma)
        model_1 = SVC(C=C, kernel=kernel, decision_function_shape='ovo', degree=degree, gamma=gamma)
        model_2 = SVC(C=C, kernel=kernel, decision_function_shape='ovo', degree=degree, gamma=gamma)
        # Fit model
        clf_0 = model_0.fit(self.X_train, y_train_0)
        clf_1 = model_1.fit(self.X_train, y_train_1)
        clf_2 = model_2.fit(self.X_train, y_train_2)

        # Calculate training error
        train_predict = ovr_by_svc(self.X_train, clf_0, clf_1, clf_2)
        self.train_errors = get_error(train_predict, self.Y_train)
        # Calculate test error
        test_predict = ovr_by_svc(self.X_test, clf_0, clf_1, clf_2)
        self.test_errors = get_error(test_predict, self.Y_test)

        # Get support vector indices
        support_vectors_0 = clf_0.support_.tolist()
        support_vectors_1 = clf_1.support_.tolist()
        support_vectors_2 = clf_2.support_.tolist()
        # self.support_indices = list(clf_0.support_.tolist() + clf_1.support_.tolist() + clf_2.support_.tolist())

        # Get w and b
        if (kernel == "linear"):
            w_0, w_1, w_2 = clf_0.coef_, clf_1.coef_, clf_2.coef_
        b_0, b_1, b_2 = clf_0.intercept_, clf_1.intercept_, clf_2.intercept_

        if (kernel == "linear"):
            return self.train_errors, self.test_errors, b_0, support_vectors_0, b_1, \
                support_vectors_1, b_2, support_vectors_2, w_0, w_1, w_2  
        else:
            return self.train_errors, self.test_errors, b_0, support_vectors_0, b_1, \
            support_vectors_1, b_2, support_vectors_2

    def SVM_slack(self, C):

        train_errors, test_errors, b_0, support_vector_0, \
            b_1, support_vector_1, b_2, support_vector_2, w_0, w_1, w_2 = svm.SVM(C=C)  
        return train_errors, test_errors, b_0, support_vector_0, \
            b_1, support_vector_1, b_2, support_vector_2, w_0, w_1, w_2


    def SVM_kernel_poly2(self, C, kernel, degree):

        train_errors, test_errors, b_0, support_vector_0, \
            b_1, support_vector_1, b_2, support_vector_2 = svm.SVM(C=C, kernel=kernel, degree=degree)  
        return train_errors, test_errors, b_0, support_vector_0, \
            b_1, support_vector_1, b_2, support_vector_2

    def SVM_kernel_poly3(self, C, kernel, degree):

        train_errors, test_errors, b_0, support_vector_0, \
            b_1, support_vector_1, b_2, support_vector_2 = svm.SVM(C=C, kernel=kernel, degree=degree)  
        return train_errors, test_errors, b_0, support_vector_0, \
            b_1, support_vector_1, b_2, support_vector_2

    def SVM_kernel_rbf(self, C, kernel, gamma):

        train_errors, test_errors, b_0, support_vector_0, \
            b_1, support_vector_1, b_2, support_vector_2 = svm.SVM(C=C, kernel=kernel, gamma=gamma)  
        return train_errors, test_errors, b_0, support_vector_0, \
            b_1, support_vector_1, b_2, support_vector_2

    def SVM_kernel_sigmoid(self, C, kernel, gamm):

        train_errors, test_errors, b_0, support_vector_0, \
            b_1, support_vector_1, b_2, support_vector_2 = svm.SVM(C=C, kernel=kernel, gamma=gamma)  
        return train_errors, test_errors, b_0, support_vector_0, \
            b_1, support_vector_1, b_2, support_vector_2


# Predict function for multiclass classification using svc in "linear" mode
# Use decision_function() method in 'ovo',
# the function values are proportional to the distance of the samples X to the separating hyperplane.
def ovr_by_svc(X, clf_0, clf_1, clf_2):

    result = []
    distance_0 = clf_0.decision_function(X)
    distance_1 = clf_1.decision_function(X)
    distance_2 = clf_2.decision_function(X)

    # Attribute the result with the maximum margin (distance)
    for i in range(0, len(X)):
        max_ = max(distance_0[i], distance_1[i], distance_2[i])
        if (max_ == distance_0[i]):
            result.append(0)
        elif (max_ == distance_1[i]):
            result.append(1)
        elif (max_ == distance_2[i]):
            result.append(2)
        else:
            result.append(-1)

    return result

# Calculating misclassification error
def get_error(y_predict, y):

    num = 0
    for i in range(0, len(y)):
        if y_predict[i] != y[i]:
            num += 1
    return num / len(y)

In [ ]:
# Output data to txt file in the required format
def to_txt(fname, train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2, w_0=[], w_1=[], w_2=[]):

    # To check if it is a linear classifier (output format is different)
    is_linear = True
    if (len(w_0) == 0 and len(w_1) == 0 and len(w_2) == 0):
        is_linear = False

    with open(fname,'w') as f:
        f.write(str(train_errors) + "\n")
        f.write(str(test_errors) + "\n")
        if (is_linear):
            f.write(str(w_0) + "\n")
        f.write(str(b_0) + "\n")
        f.write(str(support_vector_0) + "\n")
        if (is_linear):
            f.write(str(w_1) + "\n")
        f.write(str(b_1) + "\n")
        f.write(str(support_vector_1) + "\n")
        if (is_linear):
            f.write(str(w_2) + "\n")
        f.write(str(b_2) + "\n")
        f.write(str(support_vector_2) + "\n") 

In [ ]:
svm = SVM("train.txt", "test.txt")
svm.read_data()
# print(svm.X_train)
# print(svm.Y_train)

# 1. Using standard SVM model
train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2, w_0, w_1, w_2 = svm.SVM()
to_txt("SVM_linear.txt", train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2, w_0, w_1, w_2)

# 2. Using SVM with slack variables for C=0.1,...,1.0
for i in range(1, 11):
    slack = round(i * 0.1, 2)
    train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2, w_0, w_1, w_2 = svm.SVM_slack(C=slack)
    to_txt("SVM_slack_" + str(slack) + ".txt", train_errors, test_errors, b_0, support_vector_0, \
        b_1, support_vector_1, b_2, support_vector_2, w_0, w_1, w_2)

# 3.a. SVM with 2nd-order polynomial kernel
train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2 = svm.SVM(C=1, kernel="poly", degree=2)
to_txt("SVM_poly2.txt", train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2)

# 3.b. SVM with 3rd-order polynomial kernel
train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2 = svm.SVM(C=1, kernel="poly", degree=3)
to_txt("SVM_poly3.txt", train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2)

# 3.c. SVM with rbf kernel
train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2 = svm.SVM(C=1, kernel="rbf", gamma=1/2)
to_txt("SVM_rbf.txt", train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2)

# 3.d. SVM with sigmoid kernel
train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2 = svm.SVM(C=1, kernel="sigmoid", gamma=1/4)
to_txt("SVM_sigmoid.txt", train_errors, test_errors, b_0, support_vector_0, \
    b_1, support_vector_1, b_2, support_vector_2)